In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as onp
import matplotlib.pyplot as plt
from ridgeRegression import *
from diffevol import *
from moleculToVector import *
from thetas import *
from xyz2bat import *
from jax import jacfwd
from jax import vmap

/home/xomaiya/.conda/envs/ml/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
structs_with_forces = read_dataset('forces/gphase/t-1/11_cis-pos-1.xyz', 'forces/gphase/t-1/forces')
structs_with_forces = structs_with_forces[:100]
struct_description, amber_coeffs = get_struct_description()
dataset = get_dataset(structs_with_forces, struct_description)

In [4]:
dataset.coords.shape

(100, 57, 3)

In [5]:
thetas = {
    'bonds': amber_coeffs.bonds_zero_values,
    'angles': amber_coeffs.angles_zero_values,
    'torsions': amber_coeffs.torsions_zero_phase,
    'ns': amber_coeffs.ns,
    'q': amber_coeffs.qs,
    'sigma_for_vdw': amber_coeffs.sigma_for_vdw,
    'epsilon_for_vdw': amber_coeffs.epsilons_for_vdw
}

In [6]:
all_coords = np.array([struct.coords for struct in structs_with_forces])
None

/home/xomaiya/.conda/envs/ml/lib/python3.7/site-packages/jax/lib/xla_bridge.py:119: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [7]:
xyz2bat2constr_H_map = vmap(xyz2bat2constr_H, (0, None, None))
H = xyz2bat2constr_H_map(all_coords, struct_description, thetas)
None

In [8]:
xyz2bat2constr_HH = jacfwd(xyz2bat2constr_H)
xyz2bat2constr_HH_map = vmap(xyz2bat2constr_HH, (0, None, None))
HH_new = xyz2bat2constr_HH_map(all_coords, struct_description, thetas)#.reshape(-1, 57 * 3).T
HH_new.shape

(100, 3164, 57, 3)

In [9]:
energy = np.array([struct.energy for struct in structs_with_forces])
forces = np.array([struct.forces for struct in structs_with_forces])

In [10]:
de = DE(dataset, struct_description, amber_coeffs, energy)
l_bonds = len(amber_coeffs.bonds_zero_values)
l_angles = len(amber_coeffs.angles_zero_values)
l_torsions = len(amber_coeffs.torsions_zero_phase)
l_q = len(amber_coeffs.qs)
l_sigma = len(amber_coeffs.sigma_for_vdw)
l_epsilon = len(amber_coeffs.epsilons_for_vdw)

k = l_bonds + l_angles + l_torsions + l_q + l_sigma + l_epsilon + 1
de.run(k, 10)

92724.25
92724.25
92724.25
3956.543
3956.543
1925.0334
1925.0334
1925.0334
1925.0334
1925.0334


KeyboardInterrupt: 